# 1. Set-up

In [1]:
# import libraries
import pandas as pd

In [ ]:
# load excel file
# depth1depth2
input_file = 'master_thesis_2025/fcm_incognitive/data/input_data/raw/all_depths_fcm.xlsx'

# ONLY DEPTH 1
#input_file = 'master_thesis_2025/qa_model/results/all_depth1_cause_effect.xlsx'
df = pd.read_excel(input_file, sheet_name='remapped')

In [3]:
# drop missing rows
df = df.dropna(subset=['fcm_cause_remapped', 'fcm_effect_remapped'])

In [4]:
# extract all unique concepts
unique_concepts = pd.unique(df[['fcm_cause_remapped', 'fcm_effect_remapped']].values.ravel())

# Convert to list (optional)
unique_concepts_list = unique_concepts.tolist()

# Print the list
for concept in unique_concepts_list:
    print(concept)  

access to low-carbon energy sources
carbon price
digital infrastructure
electrification of industry
low carbon cement
low carbon infrastructure
raw material costs
adaption competition policy
eu industrial competitiveness
administrative complexity
eu cbam
eu climate policy
high energy prices
international trade
poor alignment of reporting requirements with sustainability performance
priortization of sustainable practices
regulatory certainty
regulatory simplification
streamlined european sustainability reporting standards
adoption of clean technology
alternative fuel penetration
decarbonization
recycling technology
australia carbon contract exit clause
carbon abatement contracts
availability of critical raw materials
circular economy legislation
digitalization
balkan industrial decarbonization
carbon capture technology
benchmark update rate
difficult investment climate
blockchain technology
circular supply chains
digital product passport
global supply chain complexity
supply chain deman

# 2. Data Wrangling

In [5]:
# clean the dataframe 
df = df.dropna(subset=['fcm_cause_remapped', 'fcm_effect_remapped'])
df['weight'] = pd.to_numeric(df['confidence'], errors='coerce')
df.loc[df['effect_type'] == 'decrease', 'weight'] *= -1

# 3. Node Mapping

In [7]:
# define exogenous levers
priority_nodes = {
    "regulatory certainty": "P1",
    "eu cbam": "P2",
    "carbon pricing policy": "P3",
    "climate finance": "U1",
    "international climate cooperation": "U2"
}

In [8]:
# Node Mapping: Concepts receive 'C' codes, descriptions receive 'S' codes
used_concepts = pd.Series(pd.unique(df[['fcm_cause_remapped', 'fcm_effect_remapped']].values.ravel()))
used_concepts = used_concepts.sort_values().reset_index(drop=True)

# Create concept code and description maps
concept_code_map = {name: f'C{i+1}' for i, name in enumerate(used_concepts)}

description_code_map = {}
node_description_map = {}

# Assign 'S' codes to descriptions but 'C' codes to concept nodes
for i, concept in enumerate(used_concepts):
    concept_code = f'C{i+1}'
    description_code = f'S{i+1}'
    description_code_map[concept] = description_code
    node_description_map[concept] = f"{description_code}. {concept}"

# Create a DataFrame with the mapped nodes
nodes_df = pd.DataFrame({
    'concept_node': [concept_code_map[name] for name in used_concepts],
    'description_code': [description_code_map[name] for name in used_concepts],
    'node_description': [node_description_map[name] for name in used_concepts]
})

In [9]:
# aggregate edges
df['source node'] = df['fcm_cause_remapped'].map(concept_code_map)
df['target node'] = df['fcm_effect_remapped'].map(concept_code_map)
df['lag'] = 1

# AGGREGATE BY SUM 
# aggregated_edges = (
#    df.groupby(['source node', 'target node', 'lag'], as_index=False)
#      .agg({'weight': 'sum'})
#)

# aggregate with average
# Aggregate edges by averaging weights
aggregated_edges = (
    df.groupby(['source node', 'target node', 'lag'], as_index=False)
      .agg({'weight': 'mean'})
)

# normalize weights to range [-1, 1]
#max_abs_weight = aggregated_edges['weight'].abs().max()
#if max_abs_weight != 0:
#    aggregated_edges['weight'] = aggregated_edges['weight'] / max_abs_weight

edges_df = aggregated_edges[['source node', 'target node', 'weight']]
topology_df = aggregated_edges[['source node', 'target node', 'weight', 'lag']]
#edges_df['weight'] = edges_df['weight'].round(3)

In [10]:
# define base node order template (compatible with Incognitive)
base_nodes_order_df = pd.DataFrame({
    'nodes order': nodes_df['concept_node'],
    'node description': nodes_df['node_description'],
    'initial value': 0,
    'auto weights': 0,
    'auto lags': 0
})

# 4. Define scenarios

## 4.1 Both depth scenarios

In [11]:
# 1. Global Green Accord  
bothdepth_scenarios = {
    'global_green_accord': {
        # exogenous levers
        "regulatory certainty": 1.0,
        "eu cbam": 0.9,
        "carbon pricing policy": 1.0,
        "climate finance": 1.0,
        "international climate cooperation": 1.0,

        # supporting policy & finance
        "harmonized carbon pricing": 0.9,
        "global climate ambition": 1.0,
        "eu climate ambition": 0.8,
        "global cbam adoption": 0.8,
        #"global carbon price": 0.9,
        "global climate mitigation": 1.0,
        "regulatory harmonization": 1.0,

        # technology push
        "renewable energy adoption": 0.8,
        "hydrogen-based electrification": 0.7,
        "carbon capture technology": 0.7,
        "digital infrastructure": 0.7,
        "adoption of clean technology": 0.8,
        "alternative fuel penetration": 0.9,
        
        # supply‑chain and economy
        "circular supply chains": 0.8,
        "carbon-intensive product competitiveness": -0.5,
        "carbon pricing disparities": -0.5,
        "carbon price": 0.9,
        "negative market responses to cbam": -0.3
    }, 
    'brussels_powerhouse': {
        # exogenous levers
        "regulatory certainty": 0.8,
        "eu cbam": 1.0,
        "carbon pricing policy": 1.0,
        "climate finance": 0.8,
        "international climate cooperation": 0.3,
        
        # EU policy & standards
        "renewable energy directive": 0.9,
        "cbam and ets integration": 0.9,
        "extension of the ets": 0.8,
        "ecodesign requirements": 0.9,
    
        # industry and economy
        "high production costs": 0.7,
        "policy-induced inflation": 0.6,
        "eu industrial competitiveness": 0.7,
        "high energy prices": 0.6,
        "carbon price": 0.6,
        "steel and metals action plan": 0.8,
        "carbon-intensive product competitiveness": 0.2,
        
        # technology push
        "digital infrastructure": 0.7,
        "digital data exchange": 0.7
    },
    'light_touch_markets':{
        # exogenous levers
        "regulatory certainty": -0.2,
        "eu cbam": 0.1,
        "carbon pricing policy": 0.1,
        "climate finance": 0.3,
        "international climate cooperation": 0.1,
        
        # roll‑backs & cost relief
        "fossil fuel price": 0.25,
        "difficult investment climate": 0.3,
        "investment leakage": 0.3,
        "reduced economic activity": 0.5,

        # Markets & supply
        "global trade disruptions": 0.4,
        "supply chain volatility": 0.5, 
        "market-driven sustainability actions": 0.8,

        # technology and innovation
        "technological innovation": -0.3
    },
    'tech_driven_acceleration':{
         # exogenous levers
        "regulatory certainty": 0.8,
        "eu cbam": 0.6,
        "carbon pricing policy": 0.6,
        "climate finance": 0.8,
        "international climate cooperation": 0.4,

        # tech advancements
        "investment in carbon capture technology": 1.0,
        "carbon capture technology": 1.0,
        "hydrogen-based electrification": 1.0,
        "renewable-based electrification": 1.0,
        "blockchain technology": 0.8,
        #"carbon market dynamics": 0.8,
        "technological trustworthiness": 0.8,
        
        # Digital backbone
        "digital infrastructure": 1.0,
        "digital technology uptake": 1.0,
        "digital data exchange": 1.0,

        # Cost & supply
        "availability of critical raw materials": 0.3
    },
    'fragmented_green_race': {
        # exogenous levers
        "regulatory certainty": 0.1, 
        "eu cbam": 0.2,
        "carbon pricing policy": 0.3,
        "climate finance": 0.3,
        "international climate cooperation": 0.2,
        
        # regional levers
        "regional climate solutions": 0.8,
        "uneven climate mitigation efforts": 0.9,
        "carbon pricing disparities": 0.9,
        "cbam's negative impact on developing countries": 1.0,
        #"harmonized carbon pricing": 0.3,
        "global climate ambition": 0.2,
        "global cbam adoption": 0.3,
        "global carbon price": 0.2,
 
        # economic fragmentation
        "carbon leakage": 0.6,
        "industrial delocalization": 0.7,
        "investment leakage": 0.6,
        "disruption due to the green transition": 0.5,
        "global trade disruptions": 0.5,
        "geopolitical tensions": 0.4, 

        # digital gaps
        #"digital technology uptake": 0.3,
        "poor alignment of reporting requirements with sustainability performance": 0.6
    }
}

## 4.2 One depth scenario

In [ ]:
scenarios = {
    'global_green_accord': {
        # exogenous levers
        #"regulatory certainty": 1.0,
        "EU CBAM": 1.0,
        "carbon pricing policy": 1.0,
        #"climate finance": 1.0,
        #"international climate cooperation": 1.0,

        # supporting policy & finance
        "harmonized carbon pricing": 0.9,
        "global climate ambition": 1.0,
        "EU climate ambition": 0.8,
        "green finance": 0.9,

        # technology push
        "renewable energy adoption": 0.8,
        "hydrogen-based electrification": 0.7,
        "carbon capture technology": 0.7,
        "adoption of clean technology": 0.8,

        # supply-chain enablers
        "circular supply chains": 0.8,
        "product carbon footprint": 0.7,

        # cost signals
        "carbon price": 0.9,
        "high energy prices": 0.6,
    },
    'brussels_powerhouse': {
        # exogenous levers
        #"regulatory certainty": 0.8,
        "EU CBAM": 1.0,
        "carbon pricing policy": 1.0,
        #"climate finance": 0.8,
        #"international climate cooperation": 0.1,

        # EU policy & standards
        "renewable energy directive": 0.9,
        "EU production standards": 0.8,
        "CBAM and ETS integration": 0.9,

        # finance & market
        "climate finance": 0.8,

        # technology push
        "digital infrastructure": 0.7,
        "digital technology uptake": 0.7
    },
    'light_touch_markets': {
        # exogenous levers
        #"regulatory certainty": 0.3,
        "EU CBAM": 0.1,
        "carbon pricing policy": 0.2,
        #"climate finance": 0.1,
        #"international climate cooperation": 0.1,

        # market-oriented policies
        "regulatory simplification": 0.7,
        "public subsidization of fossil fuels": 0.8,
        "low carbon infrastructure": 0.3,
        "carbon abatement contracts": 0.7,

        # economic climate
        "difficult investment climate": 0.7
    },
    'tech_driven_acceleration': {
        # exogenous levers
        #"regulatory certainty": 0.8,
        "EU CBAM": 0.6,
        "carbon pricing policy": 1.0,
        #"climate finance": 0.8,
        #"international climate cooperation": 0.4,

        # technological levers
        "hydrogen adoption": 0.8,
        "investment in carbon capture technology": 1.0,
        "hydrogen-based electrification": 1.0,
        "blockchain technology": 1.0,

        # digital infrastructure
        "digital technology uptake": 1.0,
        "renewable energy capacity": 0.8,
    },
    'fragmented_green_race': {
        # exogenous levers
        #"regulatory certainty": 0.2,
        "EU CBAM": 0.3,
        "carbon pricing policy": 0.3,
        #"climate finance": 0.3,
        #"international climate cooperation": 0.1,

        # patchy climate governance
        "regional climate solutions": 1.0,
        "carbon leakage": 0.8,
        "adoption of clean technology": 0.3,
        "CBAM's negative impact on developing countries": 1.0,

        # financial fragmentation
        "climate finance": 0.5,
        "international trade": 0.3
    }
}

# 5. Export each scenario to Excel

In [ ]:
# WITH DEBUGGING 
# Define output nodes relevant to carbon accounting, traceability, and digitization
output_names = [
    "carbon accounting",
    "digital product passport",
    "carbon reporting",
    "supply chain traceability", 
    "product carbon footprint",
    "sustainability reporting",
]

for scenario_name, initial_values in bothdepth_scenarios.items():
    # Copy the base node order for modification
    nodes_order_df = base_nodes_order_df.copy()

    print(f"\n=== Scenario: {scenario_name} ===")
    print(f"Initial Values: {initial_values}")

    # Apply scenario-specific initial values
    for name, value in initial_values.items():
        match = nodes_order_df['node description'].str.contains(fr"\. {name}$", regex=True)
        
        # Debugging: Show which names matched
        if not match.any():
            print(f"[WARNING] No match found for initial value name: '{name}'")
        
        nodes_order_df.loc[match, 'initial value'] = value
        nodes_order_df.loc[match & (value != 0), 'auto weights'] = 1

    # Debugging: Print the updated nodes order DataFrame
    print("\nUpdated Nodes Order DataFrame:")
    print(nodes_order_df[['node description', 'initial value', 'auto weights']].head(10))

    # Input nodes = where initial value > 0
    input_codes = nodes_order_df.loc[
        nodes_order_df['initial value'] > 0,
        'nodes order'
    ].tolist()

    # Debugging: Check the extracted input codes
    print(f"Input Codes: {input_codes}")

    # Output nodes = specified by name
    output_codes = []
    for name in output_names:
        if name in concept_code_map:
            code = concept_code_map[name]
            output_codes.append(code)
            print(f"[DEBUG] Mapped output name '{name}' to code '{code}'")
        else:
            print(f"[ERROR] Output name '{name}' not found in concept_code_map")

    # Debugging: Check the extracted output codes
    print(f"Output Codes: {output_codes}")

    # Adjust length to be equal
    max_len = max(len(input_codes), len(output_codes))
    input_codes += [None] * (max_len - len(input_codes))
    output_codes += [None] * (max_len - len(output_codes))

    # Create the DataFrame for input and output nodes
    input_output_df = pd.DataFrame({
        'input nodes': input_codes,
        'output nodes': output_codes
    })

    # Debugging: Check the final input-output DataFrame
    print("\nInput-Output DataFrame:")
    print(input_output_df)

    # === Export to Excel ===
    output_file = f'final_carbon_{scenario_name}_fcm.xlsx'
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        nodes_order_df.to_excel(writer, sheet_name='nodes-order', index=False)
        input_output_df.to_excel(writer, sheet_name='input-output-nodes', index=False)
        topology_df.to_excel(writer, sheet_name='fcm-topology', index=False)

    print(f"✅ Exported scenario: {scenario_name} → {output_file}")


=== Scenario: global_green_accord ===
Initial Values: {'regulatory certainty': 1.0, 'eu cbam': 0.9, 'carbon pricing policy': 1.0, 'climate finance': 1.0, 'international climate cooperation': 1.0, 'harmonized carbon pricing': 0.9, 'global climate ambition': 1.0, 'eu climate ambition': 0.8, 'global cbam adoption': 0.8, 'global climate mitigation': 1.0, 'regulatory harmonization': 1.0, 'renewable energy adoption': 0.8, 'hydrogen-based electrification': 0.7, 'carbon capture technology': 0.7, 'digital infrastructure': 0.7, 'adoption of clean technology': 0.8, 'alternative fuel penetration': 0.9, 'circular supply chains': 0.8, 'carbon-intensive product competitiveness': -0.5, 'carbon pricing disparities': -0.5, 'carbon price': 0.9, 'negative market responses to cbam': -0.3}


C:\Users\Emilia\AppData\Local\Temp\ipykernel_9232\3569538531.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  nodes_order_df.loc[match, 'initial value'] = value



Updated Nodes Order DataFrame:
                             node description  initial value  auto weights
0     S1. access to low-carbon energy sources            0.0             0
1             S2. adaption competition policy            0.0             0
2               S3. administrative complexity            0.0             0
3            S4. adoption of clean technology            0.8             1
4            S5. alternative fuel penetration            0.9             1
5   S6. australia carbon contract exit clause            0.0             0
6  S7. availability of critical raw materials            0.0             0
7       S8. balkan industrial decarbonization            0.0             0
8                   S9. benchmark update rate            0.0             0
9                  S10. blockchain technology            0.0             0
Input Codes: ['C4', 'C5', 'C15', 'C18', 'C20', 'C29', 'C31', 'C50', 'C69', 'C70', 'C86', 'C87', 'C88', 'C99', 'C104', 'C120', 'C152', 'C154', '

C:\Users\Emilia\AppData\Local\Temp\ipykernel_9232\3569538531.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  nodes_order_df.loc[match, 'initial value'] = value


✅ Exported scenario: brussels_powerhouse → final_carbon_brussels_powerhouse_fcm.xlsx

=== Scenario: light_touch_markets ===
Initial Values: {'regulatory certainty': -0.2, 'eu cbam': 0.1, 'carbon pricing policy': 0.1, 'climate finance': 0.3, 'international climate cooperation': 0.1, 'fossil fuel price': 0.25, 'difficult investment climate': 0.3, 'investment leakage': 0.3, 'reduced economic activity': 0.5, 'global trade disruptions': 0.4, 'supply chain volatility': 0.5, 'market-driven sustainability actions': 0.8, 'technological innovation': -0.3}

Updated Nodes Order DataFrame:
                             node description  initial value  auto weights
0     S1. access to low-carbon energy sources            0.0             0
1             S2. adaption competition policy            0.0             0
2               S3. administrative complexity            0.0             0
3            S4. adoption of clean technology            0.0             0
4            S5. alternative fuel penetra

C:\Users\Emilia\AppData\Local\Temp\ipykernel_9232\3569538531.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.2' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  nodes_order_df.loc[match, 'initial value'] = value
C:\Users\Emilia\AppData\Local\Temp\ipykernel_9232\3569538531.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  nodes_order_df.loc[match, 'initial value'] = value



Updated Nodes Order DataFrame:
                             node description  initial value  auto weights
0     S1. access to low-carbon energy sources            0.0             0
1             S2. adaption competition policy            0.0             0
2               S3. administrative complexity            0.0             0
3            S4. adoption of clean technology            0.0             0
4            S5. alternative fuel penetration            0.0             0
5   S6. australia carbon contract exit clause            0.0             0
6  S7. availability of critical raw materials            0.3             1
7       S8. balkan industrial decarbonization            0.0             0
8                   S9. benchmark update rate            0.0             0
9                  S10. blockchain technology            0.8             1
Input Codes: ['C7', 'C10', 'C15', 'C20', 'C31', 'C49', 'C50', 'C52', 'C69', 'C104', 'C120', 'C122', 'C152', 'C163', 'C191']
[DEBUG] Mapped outp

C:\Users\Emilia\AppData\Local\Temp\ipykernel_9232\3569538531.py:29: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  nodes_order_df.loc[match, 'initial value'] = value


✅ Exported scenario: fragmented_green_race → final_carbon_fragmented_green_race_fcm.xlsx
